In [22]:
import numpy as np
import pandas as pd
import pprint
from uncertainties import ufloat

pprint = pprint.pp
pi = np.pi
sin = np.sin
sqrt = np.sqrt
e = np.e

In [4]:
def get_norm(i, j, k):
    return sqrt(i**2 + j**2 + k**2)

def get_side_length(wl, norm, theta):
    return (wl * norm) / (2 * sin(theta))
    
def DIA_extinction_condition(i, j, k):
    SF = 1 + e**(pi * 1j * (i + j)) + e**(pi * 1j * (i + k)) + e**(pi * 1j * (j + k))
    SF += e**((pi/2)*1j * (i + j + k))  + e**((pi/2)*1j * (3*i + 3*j + k)) + e**((pi/2)*1j * (i + 3*j + 3*k)) + e**((pi/2)*1j * (3*i + j + 3*k))
    return np.abs(SF) < 10E-10

def FCC_extinction_condition(i, j, k):
    return ((i + j) % 2 == 0 and (i + k) % 2 == 1) or ((i + k) % 2 == 0 and (i + j) % 2 == 1) or ((j + k) % 2 == 0 and (i + k) % 2 == 1)

def BCC_extinction_condition(i, j, k):
    return (i + j + k) % 2 == 1

In [5]:
DIA_indices = set()
BCC_indices = set()
FCC_indices = set()
considered_norms = set()
for i in range(5):
    for j in range(5):
        for k in range(5):
            current_norm = get_norm(i, j, k)
            if current_norm in considered_norms:
                continue
            considered_norms.add(current_norm)
            if not DIA_extinction_condition(i, j, k):
                # Changing to (k, j, i) because I prefer
                # to read them this way. Permutations make no
                # difference
                DIA_indices.add((k, j, i))
            if not BCC_extinction_condition(i, j, k):
                BCC_indices.add((k, j, i))
            if not FCC_extinction_condition(i, j, k):
                FCC_indices.add((k, j, i))

dict_indices = {"FCC": FCC_indices, "BCC": BCC_indices, "DIA": DIA_indices}
for key, indices in dict_indices.items():
    indices = list(indices)
    dict_indices[key] = sorted(indices, key = lambda x: get_norm(*x))

In [6]:
def get_angle_ratios(X):
    angle_ratios = set()
    for angle1 in X:
        for angle2 in X:
            if sin(angle1) >= sin(angle2):
                continue
            angle_ratios.add((angle1, angle2, sin(angle1) / sin(angle2)))
    angle_ratios = np.array(list(angle_ratios))
    return angle_ratios

def get_norm_ratios(X):
    norm_ratios = set()
    for indices1 in X:
        for indices2 in X:
            norm1 = get_norm(*indices1)
            norm2 = get_norm(*indices2)
            if norm1 >= norm2:
                continue
            norm_ratios.add((indices1, indices2, 3*(norm1 / norm2, )))
    norm_ratios = np.array(list(norm_ratios))
    return norm_ratios

In [43]:
A_deg = np.array((21.1, 24.6, 36, 43.7))
B_deg = np.array((14.4, 20.5, 25.4, 29.8))
C_deg = np.array((21.4, 36.6, 44.5, 57.5))
A_rad = pi * A_deg / 180
B_rad = pi * B_deg / 180
C_rad = pi * C_deg / 180
A_ratios = get_angle_ratios(A_rad)
B_ratios = get_angle_ratios(B_rad)
C_ratios = get_angle_ratios(C_rad)

FCC_indices = [(1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1)]
BCC_indices = [(1, 1, 0), (2, 0, 0), (2, 2, 0), (2, 1, 1)]
DIA_indices = [(1, 1, 1), (2, 2, 0), (3, 1, 1), (4, 0, 0)]

FCC_ratios = get_norm_ratios(FCC_indices)
BCC_ratios = get_norm_ratios(BCC_indices)
DIA_ratios = get_norm_ratios(DIA_indices)

indices = {"FCC": FCC_indices, "BCC": BCC_indices, "DIA": DIA_indices}
angles_rad = {"A": A_rad, "B": B_rad, "C": C_rad}
angles_deg = {"A": A_deg, "B": B_deg, "C": C_deg}
norm_ratios = {"FCC": FCC_ratios, "BCC": BCC_ratios, "DIA": DIA_ratios}
sin_ratios = {"A": A_ratios, "B": B_ratios, "C": C_ratios}

In [44]:
proposed_pairs = [("A", "FCC"), ("B", "BCC"), ("C", "DIA")]

In [45]:
def check_pairs(crystal, lattice):
    # Finding coincidences
    coincidences = []
    for norm_ratio in norm_ratios[lattice]:
        for angle_ratio in sin_ratios[crystal]:
            if np.abs(norm_ratio[-1,-1] - angle_ratio[-1]) < 3E-3:
                coincidences.append((*norm_ratio[:-1], angle_ratio[:-1]))
    
    # Calculating a
    wl = 1.5 # Armstrong
    side_lengths = []
    for coincidence in coincidences:
        max_sin = np.max(coincidence[-1])
        max_norm = max(get_norm(*coincidence[0]), get_norm(*coincidence[1]))
        side_lengths.append(get_side_length(wl, max_norm, max_sin))
    
    side_lengths = np.array(side_lengths)
    mean, std = side_lengths.mean(), side_lengths.std()
    side_length = (mean, std, 100 * std / mean)

    sorted_indices = sorted(indices[lattice], key = lambda indices: get_norm(*indices))
    sorted_angles = sorted(angles_deg[crystal])
    angle_index_pairs = [(I, a) for I, a in zip(sorted_indices, sorted_angles)]

    return len(coincidences), side_length, angle_index_pairs

In [52]:
data = {"Cristal - Red": [], "Lado $a$ (\r{A})": [], "Ángulo - Índices 1": [],
        "Ángulo - Índices 2": [],  "Ángulo - Índices 3": [], "Ángulo - Índices 4": []}

for crystal, lattice in proposed_pairs:
    coincidences, side_length, angle_index_pairs = check_pairs(crystal, lattice)
    data["Cristal - Red"].append(f"{crystal} - {lattice}")
    data["Lado $a$ (\r{A})"].append(ufloat(side_length[0], side_length[1]))
    for angle_index_pair, angle_number in zip(angle_index_pairs, [1, 2, 3, 4]):
        tuple = [str(i) for i in angle_index_pair[0]]
        data[f"Ángulo - Índices {angle_number}"].append(f"{angle_index_pair[1]} - {''.join(tuple)}")

df = pd.DataFrame(data).T

In [53]:
print(df.to_latex())

\begin{tabular}{llll}
\toprule
 & 0 & 1 & 2 \\
\midrule
Cristal - Red & A - FCC & B - BCC & C - DIA \\
{A}) & 3.604+/-0.004 & 4.1+/-0.5 & 3.554+/-0.004 \\
Ángulo - Índices 1 & 21.1 - 111 & 14.4 - 110 & 21.4 - 111 \\
Ángulo - Índices 2 & 24.6 - 200 & 20.5 - 200 & 36.6 - 220 \\
Ángulo - Índices 3 & 36.0 - 220 & 25.4 - 211 & 44.5 - 311 \\
Ángulo - Índices 4 & 43.7 - 311 & 29.8 - 220 & 57.5 - 400 \\
\bottomrule
\end{tabular}

